This is a test to validate the commands in python before we move to JS

#https://duckdb.org/docs/guides/python/jupyter.html

In [ ]:
#%pip install duckdb
#%pip install pandas
#%pip install jupysql matplotlib duckdb-engine
#%pip install -U sentence-transformers
#%pip install -U numpy==1.26

In [1]:
import duckdb
import pandas as pd

%load_ext sql


In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

conn = duckdb.connect(":memory:")
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db


In [ ]:
# conn.execute("CREATE TABLE t1 (id INTEGER)")
conn.execute("INSERT INTO t1 VALUES (1)")
conn.execute("SELECT * FROM t1")
result = conn.fetchall()
# print result
print(result)



In [7]:
%sql duckdb:///:memory:

In [ ]:
%sql SELECT 'Off and flying!' AS a_duckdb_column;

In [ ]:
%%sql
SELECT
    schema_name,
    function_name
FROM duckdb_functions()
ORDER BY ALL DESC
LIMIT 5;

In [ ]:
# Install and load the vss extension
%%sql
INSTALL vss;
LOAD vss;

In [ ]:
%sql CREATE TABLE test (id INTEGER, name STRING);
%sql INSERT INTO test VALUES (1, 'Alice');
%sql SELECT * FROM test

In [ ]:
%sql SELECT * FROM duckdb_tables

In [ ]:
%sql CREATE TABLE my_vector_table (id INTEGER, vec FLOAT[384]);

In [67]:
from sentence_transformers import SentenceTransformer
# sentences = ["I like hot dogs", "The submarine is in the ocean", "sandwich", "Boat"]
sentences = ["Este e um cachorro quente", "Um submarino submergiu na agua", "comida", "navio"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings.shape)


(4, 384)


In [68]:
conn.execute("DROP TABLE IF EXISTS my_vector_table;")
conn.execute("CREATE TABLE my_vector_table (id INTEGER, vec FLOAT[384])")

In [69]:
conn.execute("INSERT INTO my_vector_table VALUES (?,?)", (0,embeddings[0].tolist(),))
conn.execute("INSERT INTO my_vector_table VALUES (?,?)", (1,embeddings[1].tolist(),))



In [70]:
conn.execute("INSTALL vss;")
conn.execute("LOAD vss;")
conn.execute("SET hnsw_enable_experimental_persistence = true")
conn.execute("DROP INDEX IF EXISTS my_hnsw_index")
conn.execute("CREATE INDEX my_hnsw_index ON my_vector_table USING HNSW (vec)")


In [72]:
result = conn.execute(
    "SELECT id FROM my_vector_table ORDER BY array_distance(vec, ?::FLOAT[384]) LIMIT 3",
    (embeddings[3].tolist(),)
)
print(result.fetchall())


[(0,), (1,)]
